In [1]:
# ===================================
# YASSIR MAIN APP - CUSTOMER REVIEWS
# Complete Data Collection
# ===================================

from google_play_scraper import Sort, reviews, reviews_all
import pandas as pd
import json
from datetime import datetime
import os

print("✅ Libraries imported successfully!")
print(f"📅 Collection started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()
print("📱 Target: Yassir - Ride, Eat & Shop (Main Customer App)")
print("🎯 Goal: Collect ALL available reviews")

✅ Libraries imported successfully!
📅 Collection started: 2026-02-27 18:07:10

📱 Target: Yassir - Ride, Eat & Shop (Main Customer App)
🎯 Goal: Collect ALL available reviews


In [2]:
# ===================================
# COLLECT ALL AVAILABLE REVIEWS
# ===================================

print("🔍 Collecting ALL reviews from Yassir main app...")
print("⏳ This may take 5-10 minutes (depending on total reviews)...")
print("   Please be patient...")
print()

try:
    # Use reviews_all() to get ALL reviews (not limited to 400)
    result = reviews_all(
        'com.yatechnologies.yassir_rider',  # Main customer app
        sleep_milliseconds=0,  # No delay between requests
        lang='en',
        country='dz',
        sort=Sort.NEWEST,
    )
    
    print()
    print("=" * 80)
    print("✅ COLLECTION SUCCESSFUL!")
    print("=" * 80)
    print()
    print(f"📊 Total reviews collected: {len(result)}")
    
    if len(result) > 0:
        print(f"📅 Date range:")
        print(f"   • Newest: {result[0]['at'].strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   • Oldest: {result[-1]['at'].strftime('%Y-%m-%d %H:%M:%S')}")
        print()
        print(f"📝 Sample review:")
        print(f"   • Author: {result[0]['userName']}")
        print(f"   • Rating: {'⭐' * result[0]['score']} ({result[0]['score']}/5)")
        print(f"   • Text: {result[0]['content'][:150]}...")
    
except Exception as e:
    print()
    print(f"❌ Error occurred: {e}")
    print()
    print("Trying with limited collection (2000 reviews)...")
    
    # Fallback: collect maximum possible
    try:
        result, _ = reviews(
            'com.yatechnologies.yassir_rider',
            lang='en',
            country='dz',
            sort=Sort.NEWEST,
            count=2000  # Maximum
        )
        print(f"✅ Collected {len(result)} reviews (fallback method)")
        
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")
        result = []

print()
print("=" * 80)

🔍 Collecting ALL reviews from Yassir main app...
⏳ This may take 5-10 minutes (depending on total reviews)...
   Please be patient...


✅ COLLECTION SUCCESSFUL!

📊 Total reviews collected: 6120
📅 Date range:
   • Newest: 2026-02-25 23:00:52
   • Oldest: 2017-09-22 20:56:10

📝 Sample review:
   • Author: A Google user
   • Rating: ⭐⭐⭐ (3/5)
   • Text: bien...



In [3]:
# ===================================
# STRUCTURE DATA WITH COMPLETE METADATA
# ===================================

print("📦 Structuring data with metadata...")
print()

structured_data = []

for idx, review in enumerate(result):
    structured_data.append({
        # ===== IDENTIFIERS =====
        'id': idx + 1,
        'review_id': review.get('reviewId', ''),
        
        # ===== CONTENT METADATA =====
        'text': review['content'],
        'text_length': len(review['content']),
        'word_count': len(review['content'].split()),
        
        # ===== TEMPORAL METADATA =====
        'review_date': review['at'].strftime('%Y-%m-%d'),
        'review_time': review['at'].strftime('%H:%M:%S'),
        'review_datetime': review['at'].strftime('%Y-%m-%d %H:%M:%S'),
        'review_year': review['at'].year,
        'review_month': review['at'].month,
        'collection_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        
        # ===== SOURCE METADATA =====
        'source': 'Google Play Store',
        'user_type': 'Customer',  # Customers/Riders (NOT drivers)
        'platform': 'Android',
        'company': 'Yassir',
        'app_name': 'Yassir - Ride, Eat & Shop',
        'app_id': 'com.yatechnologies.yassir_rider',
        'country': 'Algeria',
        
        # ===== AUTHOR METADATA =====
        'author': review['userName'],
        'author_image': review.get('userImage', ''),
        
        # ===== RATING METADATA =====
        'rating': review['score'],
        'thumbs_up_count': review.get('thumbsUpCount', 0),
        
        # ===== COLLECTION METADATA =====
        'collection_method': 'google-play-scraper API (reviews_all)',
        'language': 'mixed (fr/ar/en)',
    })

print(f"✅ Successfully structured {len(structured_data)} reviews!")
print()
print("📋 Metadata fields per review:")
print("   • Content: text, length, word count")
print("   • Temporal: date, time, year, month")
print("   • Source: platform, user type, app details")
print("   • Author: name, image URL")
print("   • Rating: score (1-5), thumbs up count")
print("   • Collection: method, language, collection date")

📦 Structuring data with metadata...

✅ Successfully structured 6120 reviews!

📋 Metadata fields per review:
   • Content: text, length, word count
   • Temporal: date, time, year, month
   • Source: platform, user type, app details
   • Author: name, image URL
   • Rating: score (1-5), thumbs up count
   • Collection: method, language, collection date


In [4]:
# ===================================
# CONVERT TO DATAFRAME & ANALYZE
# ===================================

df = pd.DataFrame(structured_data)

print("=" * 80)
print("📊 YASSIR CUSTOMER REVIEWS - COMPLETE DATASET")
print("=" * 80)
print()

# ===== BASIC STATISTICS =====
print("📈 COLLECTION STATISTICS:")
print(f"   • Total reviews collected: {len(df):,}")
print(f"   • Date range: {df['review_date'].min()} to {df['review_date'].max()}")
print(f"   • Time span: {(pd.to_datetime(df['review_date'].max()) - pd.to_datetime(df['review_date'].min())).days} days")
print(f"   • Average rating: {df['rating'].mean():.2f}/5 ⭐")
print(f"   • Average text length: {df['text_length'].mean():.0f} characters")
print(f"   • Average word count: {df['word_count'].mean():.0f} words")
print()

# ===== RATING DISTRIBUTION =====
print("⭐ RATING DISTRIBUTION:")
for rating in [5, 4, 3, 2, 1]:
    count = len(df[df['rating'] == rating])
    percentage = (count / len(df)) * 100
    stars = '⭐' * rating
    bar = '█' * int(percentage / 2)
    print(f"   {stars} ({rating}): {count:5d} reviews ({percentage:5.1f}%) {bar}")

print()

# ===== SENTIMENT BREAKDOWN =====
positive = len(df[df['rating'] >= 4])
neutral = len(df[df['rating'] == 3])
negative = len(df[df['rating'] <= 2])

print("😊 SENTIMENT BREAKDOWN:")
print(f"   • Positive (4-5 stars): {positive:5d} ({positive/len(df)*100:5.1f}%)")
print(f"   • Neutral (3 stars):    {neutral:5d} ({neutral/len(df)*100:5.1f}%)")
print(f"   • Negative (1-2 stars): {negative:5d} ({negative/len(df)*100:5.1f}%)")
print()

# ===== YEARLY DISTRIBUTION =====
print("📅 REVIEWS BY YEAR:")
yearly = df['review_year'].value_counts().sort_index()
for year, count in yearly.items():
    print(f"   • {year}: {count:5d} reviews")
print()

# ===== TOP REVIEWERS =====
print("👤 TOP 5 REVIEWERS (by thumbs up):")
top_reviews = df.nlargest(5, 'thumbs_up_count')[['author', 'rating', 'thumbs_up_count', 'text']]
for idx, row in top_reviews.iterrows():
    print(f"   • {row['author']}: {'⭐' * row['rating']} ({row['thumbs_up_count']} 👍)")
    print(f"     {row['text'][:100]}...")
    print()

print("=" * 80)

📊 YASSIR CUSTOMER REVIEWS - COMPLETE DATASET

📈 COLLECTION STATISTICS:
   • Total reviews collected: 6,120
   • Date range: 2017-09-22 to 2026-02-25
   • Time span: 3078 days
   • Average rating: 3.24/5 ⭐
   • Average text length: 46 characters
   • Average word count: 8 words

⭐ RATING DISTRIBUTION:
   ⭐⭐⭐⭐⭐ (5):  2874 reviews ( 47.0%) ███████████████████████
   ⭐⭐⭐⭐ (4):   528 reviews (  8.6%) ████
   ⭐⭐⭐ (3):   247 reviews (  4.0%) ██
   ⭐⭐ (2):   147 reviews (  2.4%) █
   ⭐ (1):  2324 reviews ( 38.0%) ██████████████████

😊 SENTIMENT BREAKDOWN:
   • Positive (4-5 stars):  3402 ( 55.6%)
   • Neutral (3 stars):      247 (  4.0%)
   • Negative (1-2 stars):  2471 ( 40.4%)

📅 REVIEWS BY YEAR:
   • 2017:    17 reviews
   • 2018:    83 reviews
   • 2019:   101 reviews
   • 2020:    92 reviews
   • 2021:   488 reviews
   • 2022:   795 reviews
   • 2023:   895 reviews
   • 2024:  2200 reviews
   • 2025:  1331 reviews
   • 2026:   118 reviews

👤 TOP 5 REVIEWERS (by thumbs up):
   • knitella b

In [5]:
# ===================================
# SAMPLE REVIEWS FROM EACH CATEGORY
# ===================================

print("📝 SAMPLE REVIEWS:")
print("=" * 80)
print()

# Positive (5 stars)
if len(df[df['rating'] == 5]) > 0:
    positive_review = df[df['rating'] == 5].iloc[0]
    print("✅ POSITIVE REVIEW (5 stars)")
    print("-" * 80)
    print(f"Author: {positive_review['author']}")
    print(f"Date: {positive_review['review_date']}")
    print(f"Rating: {'⭐' * 5}")
    print(f"Thumbs up: 👍 {positive_review['thumbs_up_count']}")
    print(f"Text:\n{positive_review['text']}")
    print()

# Neutral (3 stars)
if len(df[df['rating'] == 3]) > 0:
    neutral_review = df[df['rating'] == 3].iloc[0]
    print("😐 NEUTRAL REVIEW (3 stars)")
    print("-" * 80)
    print(f"Author: {neutral_review['author']}")
    print(f"Date: {neutral_review['review_date']}")
    print(f"Rating: {'⭐' * 3}")
    print(f"Thumbs up: 👍 {neutral_review['thumbs_up_count']}")
    print(f"Text:\n{neutral_review['text']}")
    print()

# Negative (1 star)
if len(df[df['rating'] == 1]) > 0:
    negative_review = df[df['rating'] == 1].iloc[0]
    print("❌ NEGATIVE REVIEW (1 star)")
    print("-" * 80)
    print(f"Author: {negative_review['author']}")
    print(f"Date: {negative_review['review_date']}")
    print(f"Rating: {'⭐' * 1}")
    print(f"Thumbs up: 👍 {negative_review['thumbs_up_count']}")
    print(f"Text:\n{negative_review['text']}")
    print()

print("=" * 80)

📝 SAMPLE REVIEWS:

✅ POSITIVE REVIEW (5 stars)
--------------------------------------------------------------------------------
Author: A Google user
Date: 2026-02-24
Rating: ⭐⭐⭐⭐⭐
Thumbs up: 👍 0
Text:
Appuyez de manière prolongée sur un élément pour l'épingler. Les éléments retirés seront supprimés au bout d'une heure.

😐 NEUTRAL REVIEW (3 stars)
--------------------------------------------------------------------------------
Author: A Google user
Date: 2026-02-25
Rating: ⭐⭐⭐
Thumbs up: 👍 0
Text:
bien

❌ NEGATIVE REVIEW (1 star)
--------------------------------------------------------------------------------
Author: A Google user
Date: 2026-02-24
Rating: ⭐
Thumbs up: 👍 0
Text:
اسعار غالية فوق اللازم



In [6]:
# ===================================
# SAVE DATA TO FILES
# ===================================

print("💾 Saving data to files...")
print()

# Create directories
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

# ===== SAVE AS JSON (Raw format with all details) =====
json_path = '../data/raw/yassir_customers_raw.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=2)

json_size = os.path.getsize(json_path) / 1024 / 1024  # MB

print(f"✅ JSON saved: {json_path}")
print(f"   • Format: JSON (structured data)")
print(f"   • Size: {json_size:.2f} MB")
print(f"   • Records: {len(structured_data):,}")
print()

# ===== SAVE AS CSV (Processed format for analysis) =====
csv_path = '../data/processed/yassir_customers_processed.csv'
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

csv_size = os.path.getsize(csv_path) / 1024 / 1024  # MB

print(f"✅ CSV saved: {csv_path}")
print(f"   • Format: CSV (easy to open in Excel/Pandas)")
print(f"   • Size: {csv_size:.2f} MB")
print(f"   • Records: {len(df):,}")
print(f"   • Columns: {len(df.columns)}")
print()

print("=" * 80)
print("🎉 DATA COLLECTION COMPLETE!")
print("=" * 80)
print()
print(f"📊 Summary:")
print(f"   • Total customer reviews: {len(df):,}")
print(f"   • Average rating: {df['rating'].mean():.2f}/5")
print(f"   • Date range: {df['review_date'].min()} to {df['review_date'].max()}")
print(f"   • Positive reviews: {len(df[df['rating'] >= 4]):,} ({len(df[df['rating'] >= 4])/len(df)*100:.1f}%)")
print(f"   • Negative reviews: {len(df[df['rating'] <= 2]):,} ({len(df[df['rating'] <= 2])/len(df)*100:.1f}%)")
print()
print(f"📁 Files saved:")
print(f"   1. {json_path}")
print(f"   2. {csv_path}")
print()
print("✅ Ready for Person 2 to create visualizations!")
print("✅ Ready for analysis and presentation!")
print()
print("=" * 80)

💾 Saving data to files...

✅ JSON saved: ../data/raw/yassir_customers_raw.json
   • Format: JSON (structured data)
   • Size: 5.69 MB
   • Records: 6,120

✅ CSV saved: ../data/processed/yassir_customers_processed.csv
   • Format: CSV (easy to open in Excel/Pandas)
   • Size: 2.74 MB
   • Records: 6,120
   • Columns: 24

🎉 DATA COLLECTION COMPLETE!

📊 Summary:
   • Total customer reviews: 6,120
   • Average rating: 3.24/5
   • Date range: 2017-09-22 to 2026-02-25
   • Positive reviews: 3,402 (55.6%)
   • Negative reviews: 2,471 (40.4%)

📁 Files saved:
   1. ../data/raw/yassir_customers_raw.json
   2. ../data/processed/yassir_customers_processed.csv

✅ Ready for Person 2 to create visualizations!
✅ Ready for analysis and presentation!

